# Lesson 22: Implementing the Foundations of the Writing Workflow

In this lesson, we'll dive deep into the Brown writing agent, a sophisticated system designed to generate high-quality technical articles about AI.

**Learning Objectives:**

- Understand context engineering techniques for manipulating multiple inputs, structure and reason accordingly
- Learn the orchestrator-worker pattern for generating media assets like Mermaid diagrams
- Entity modeling using Pydantic to structure guidelines, research, profiles, and media
- Writing professional documents such as articles or lessons


> [!NOTE]
> 💡 Remember that you can also run `brown` as a standalone Python package by going to `lessons/writing_workflow/` and following the instructions from there.

## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:


In [1]:
%load_ext autoreload
%autoreload 2

### Set Up Python Environment

To set up your Python virtual environment using `uv` and load it into the Notebook, follow the step-by-step instructions from the `Course Admin` lesson from the beginning of the course.

**TL/DR:** Be sure the correct kernel pointing to your `uv` virtual environment is selected.


### Configure Gemini API

To configure the Gemini API, follow the step-by-step instructions in the `Course Admin` lesson.

Here is a quick checklist of what you need to run this notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/api-keys).
2.  From the root of your project, run: `cp .env.example .env` 
3.  Within the `.env` file, fill in the `GOOGLE_API_KEY` variable:

Now, the code below will load the key from the `.env` file:

In [2]:
from utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])

Environment variables loaded from `/Users/pauliusztin/Documents/01_projects/TAI/agentic-ai-engineering-course/.env`
Environment variables loaded successfully.


### Import Key Packages


In [3]:
import nest_asyncio
from utils import pretty_print

nest_asyncio.apply()  # Allow nested async usage in notebooks

### Download Required Files

We need to download the configuration files and input data that Brown uses for article generation and editing.

First, let's download the configs folder:

In [4]:
%%capture

!rm -rf configs
!curl -L -o configs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/configs.zip
!unzip configs.zip
!rm -rf configs.zip

Now, let's download the inputs folder containing profiles, examples, and test data:

In [5]:
%%capture

!rm -rf inputs
!curl -L -o inputs.zip https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/data/inputs.zip
!unzip inputs.zip
!rm -rf inputs.zip

Let's verify what we downloaded:

In [6]:
%ls

article_guideline.md   inputs/                notebook_guideline.md
configs/               notebook.ipynb


### Set Up Directory Constants

Now let's define constants to reference these directories throughout the notebook:

In [7]:
from pathlib import Path

CONFIGS_DIR = Path("configs")
INPUTS_DIR = Path("inputs")

print(f"Configs directory exists: {CONFIGS_DIR.exists()}")
print(f"Inputs directory exists: {INPUTS_DIR.exists()}")

Configs directory exists: True
Inputs directory exists: True


In [8]:
EXAMPLES_DIR = Path("inputs/examples/course_lessons")
PROFILES_DIR = Path("inputs/profiles")

print(f"Examples directory exists: {EXAMPLES_DIR.exists()}")
print(f"Profiles directory exists: {PROFILES_DIR.exists()}")

Examples directory exists: True
Profiles directory exists: True


First, we will load a simpler example that runs faster and is easier to understand. At the end, we will load a larger sample that is closer to what we do on our end to generate professional articles:

In [9]:
SAMPLE_DIR = Path("inputs/tests/01_sample_small")

print(f"Samples directory exists: {SAMPLE_DIR.exists()}")

Samples directory exists: True


### Understanding the Brown Package

Throughout this notebook, we'll be importing code from the `brown` package. This package contains all the code for the writing workflow which is located in the `lessons/writing_workflow/`. 

The Brown package is installed as a local Python package using `uv` and contains all the core functionality we'll explore:

- **Entities** (`brown.entities`): Pydantic models for articles, guidelines, research, profiles, and media
- **Nodes** (`brown.nodes`): Workflow components like ArticleWriter and MediaGeneratorOrchestrator
- **Models** (`brown.models`): LLM configuration and initialization utilities
- **Loaders** (`brown.loaders`): Classes to load markdown content into entities
- **Builders** (`brown.builders`): Factory patterns for creating loaders, renderers, and models
- **Renderers** (`brown.renderers`): Classes to save entities back to markdown files
- **Config** (`brown.config`): Settings management using Pydantic

Let's start by understanding how we load environment variables and configure the system.

> [!NOTE]
> 💡 You can also run `brown` as a standalone Python package by going to `lessons/writing_workflow/` and following the instructions from there.


## 2. Implementing Our Settings Class

The Brown agent uses a centralized settings system built with Pydantic's `BaseSettings` to load all sensitive credentials, such as API Keys, from a centralized class. This ensures that all your credentials are stored within a local `.env` file or in memory, while it leverages all the type safety features that come with Pydantic.

Let's examine the `Settings` class from `brown.config`:

```python
import os
from functools import lru_cache
from typing import Annotated

from loguru import logger
from pydantic import Field, FilePath, SecretStr
from pydantic_settings import BaseSettings, SettingsConfigDict

ENV_FILE_PATH = os.getenv("ENV_FILE_PATH", ".env")
logger.info(f"Loading environment file from `{ENV_FILE_PATH}`")


class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file=ENV_FILE_PATH, extra="ignore", env_file_encoding="utf-8")

    # --- Gemini ---
    GOOGLE_API_KEY: SecretStr | None = Field(default=None, description="The API key for the Gemini API.")

    # --- Opik ---
    OPIK_ENABLED: bool = Field(default=False, description="Whether to use Opik for monitoring and logging.")
    OPIK_WORKSPACE: str | None = Field(default=None, description="Name of the Opik workspace containing the project.")
    OPIK_PROJECT_NAME: str = Field(default="brown", description="Name of the Opik project.")
    OPIK_API_KEY: SecretStr | None = Field(default=None, description="The API key for the Opik API.")

    # --- App Config ---
    CONFIG_FILE: Annotated[FilePath, Field(default="configs/course.yaml", description="Path to the application configuration YAML file.")]


@lru_cache(maxsize=1)
def get_settings() -> Settings:
    return Settings()
```

When the `Settings` class is instantiated, it first looks for environment variables in memory, then in the `.env` file. Thus, while working with these Notebooks, it's enough to have them only inside the memory, which is taken care of by the `env.load` function from the setup section.

But if you run the Notebook locally, you can also keep them inside a local `.env` file. The choice is yours. 

**Key Design Patterns:**

1. **Singleton Pattern**: The `@lru_cache(maxsize=1)` decorator ensures we only instantiate `Settings` once, making it behave like a singleton throughout the application.

2. **Type Safety**: Using `SecretStr` for sensitive data like API keys ensures they're handled securely and not accidentally logged.

3. **Environment Integration**: `SettingsConfigDict` automatically loads values from the `.env` file, with sensible defaults for missing values.

4. **Flexible Configuration**: The `CONFIG_FILE` field points to a YAML configuration that controls workflow behavior, model selection, and more. More on this in Lesson 23.

## 3. Scoping the Writing Workflow Architecture

Before diving into the implementation details, let's understand the three-step workflow that powers the Brown writing agent.

### The Three-Step Process

**Step 1: Load Context into Memory**

The first step involves gathering all the necessary context that will guide the article generation process:

- **Article Guideline**: The user's input describing what the article should contain, its outline, and any specific requirements
- **Research**: Factual data, references, and information that will support the article's claims
- **Few-Shot Examples**: Sample articles demonstrating the desired writing style and structure (your training set)
- **Content Generation Profiles**: Specialized profiles that control different aspects of the writing:
  - Character profile (voice and perspective)
  - Article profile (article-specific rules)
  - Structure profile (formatting rules)
  - Mechanics profile (writing mechanics)
  - Terminology profile (word choice)
  - Tonality profile (tone and style)

**Step 2: Generate Media Items (Orchestrator-Worker Pattern)**

Once we have the context, based on the article guideline, we need to generate any required media assets like diagrams or charts. This step uses the orchestrator-worker pattern:

- The **MediaGeneratorOrchestrator** analyzes the article guideline and research to identify what media items need to be generated
- For each identified requirement, the orchestrator delegates to specialized **worker tools** (e.g., `MermaidDiagramGenerator`)
- Workers generate their specialized content type in parallel
- All generated media items are collected and prepared for integration into the article

**Step 3: Write the Article**

With all context and media items ready, the **ArticleWriter** node generates the final article:

- Takes all the context from Step 1
- Integrates the media items from Step 2
- Follows all profile rules to generate content matching the desired style
- Produces a complete, high-quality article

Let's visualize this workflow with a diagram:

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/l22_writing_workflow.png" alt="Article Generation Workflow"/>

Now that we understand the high-level workflow, let's dive into the implementation details of each component, starting with how we structure and load the context.

## 4. Modeling our Domain Entities as Context for LLMs

Now that we understand the high-level workflow, let's explore how we model and load the various context components. 

The Brown writing workflow uses Pydantic models (entities) to represent different types of content, and specialized loaders to read this content from disk. 

Everything within the domain layer, such as the article guideline, research, writing profiles or few-shot examples, is modeled as an entity. Like this, we can very easily attach functionality to each type, structure it and add data validation. As we presented in the project structure lesson, these entities will be passed all over the AI application. They will be used as inputs and outputs to all our business logic. They will be the only way to pass information between functions, classes and between LLMs (software 3.0) and our Python code (software 1.0).


### The ContextMixin: Foundation for Context Engineering

Before we dive into specific entities, let's understand the `ContextMixin` - a crucial abstraction that enables powerful context engineering throughout the system.

The `ContextMixin` provides a standardized way to convert any entity into a context representation surrounded by XML tags. This is essential for:

1. **Clear Boundaries**: XML tags clearly delineate different pieces of context in prompts
2. **Structured Prompts**: LLMs can easily parse and understand structured XML context
3. **Consistent Interface**: All entities follow the same pattern for converting Python objects to LLM input context
4. **Easy Composition**: Different context elements can be seamlessly combined

Here's the implementation from `brown.entities.mixins`:

```python
from abc import abstractmethod
from brown.utils.s import camel_to_snake


class ContextMixin:
    @property
    def xml_tag(self) -> str:
        return camel_to_snake(self.__class__.__name__)

    @abstractmethod
    def to_context(self) -> str:
        """Context representation of the object."""
        pass
```

**Key Features:**

- **Automatic Tag Generation**: The `xml_tag` property automatically converts the class name to snake_case (e.g., `ArticleGuideline` → `article_guideline`)
- **Abstract Method**: `to_context()` must be implemented by each entity, ensuring consistency
- **Simplicity**: The `ContextMixin` is a simple interface, that standardizes how we map Python objects to LLM context inputs

For example, an `ArticleGuideline` entity will be wrapped in `<article_guideline>...</article_guideline>` tags when passed to the LLM.


### Article Guideline Entity

The `ArticleGuideline` represents the user's input - what they want the article to contain, how it should be structured, and any specific requirements. It's the primary driver of content generation that is different for each article.

From `brown.entities.guidelines`:
```python
from pydantic import BaseModel
from brown.entities.mixins import ContextMixin


class ArticleGuideline(BaseModel, ContextMixin):
    content: str

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    <content>{self.content}</content>
</{self.xml_tag}>
"""

    def __str__(self) -> str:
        return f"ArticleGuideline(len_content={len(self.content)})"
```

⚠️ Note how we implemented the `to_context()` method.

The guideline typically contains:
- Article outline and section structure
- Specific instructions for each section
- Length constraints or requirements
- Important references to prioritize
- Any special formatting needs

Let's load the sample article guideline:


In [10]:
from brown.loaders import MarkdownArticleGuidelineLoader

guideline_loader = MarkdownArticleGuidelineLoader(uri=Path("article_guideline.md"))
article_guideline = guideline_loader.load(working_uri=SAMPLE_DIR)


2025-11-29 17:55:09.155 | INFO     | brown.config:<module>:10 - Loading environment file from `.env`


We will show only the first 1200 characters to see how the enclosing XML tags logic works:

In [11]:
cropped_article_guideline = article_guideline.model_copy()
cropped_article_guideline.content = f"{cropped_article_guideline.content[:1200]}..."

pretty_print.wrapped(f"{cropped_article_guideline.content}", title="Article Guideline (First 1200 Chars)", width=150)

-------------------------------------------------------- Article Guideline (First 1200 Chars) --------------------------------------------------------
  ## Outline

1. Introduction: The Critical Decision Every AI Engineer Faces
2. Understanding the Spectrum: From Workflows to Agents
3. Choosing Your Path
4. Conclusion: The Challenges of Every AI Engineer

## Section 1 - Introduction: The Critical Decision Every AI Engineer Faces

- **The Problem:** When building AI applications, engineers face a critical architectural decision early in their development process. Should they create a predictable, step-by-step workflow where they control every action, or should they build an autonomous agent that can think and decide for itself? This is one of the key decisions that will impact everything from the product such as development time and costs to reliability and user experience.
- Quick walkthrough of what we'll learn by the end of this lesson

- **Section length:** 100 words

## Section 2 -

Let's also call the `to_context()` method:

In [12]:
pretty_print.wrapped(f"{cropped_article_guideline.to_context()}", title="Article Guideline as Context")

----------------------------------- Article Guideline as Context -----------------------------------
  
<article_guideline>
    <content>## Outline

1. Introduction: The Critical Decision Every AI Engineer Faces
2. Understanding the Spectrum: From Workflows to Agents
3. Choosing Your Path
4. Conclusion: The Challenges of Every AI Engineer

## Section 1 - Introduction: The Critical Decision Every AI Engineer Faces

- **The Problem:** When building AI applications, engineers face a critical architectural decision early in their development process. Should they create a predictable, step-by-step workflow where they control every action, or should they build an autonomous agent that can think and decide for itself? This is one of the key decisions that will impact everything from the product such as development time and costs to reliability and user experience.
- Quick walkthrough of what we'll learn by the end of this lesson

- **Section length:** 100 words

## Section 2 - Understanding t

Note how the whole content is surrounded by the `<article_gudeline>...</article_guideline>` XML tags, while the content attribute is surrounded by the `<content>...</content>` XML tag.

We could always pass the `content` field directly as a plain string. If you want to add only one or two entities within the context, that would have worked. But in our use case, we will model up to 10 different context elements, each with its own attributes. Thus, having a clear way to tell the LLM how to differentiate between them, and how to reference them when writing instructions it's an essential skill for context engineering. 

The `to_context()` method maps the Pydantic model to its XML representation, where the root XML tag is inferred from the entity class name, and the fields use the attribute's name directly. Like this, when we pass these entities to an LLM, it can clearly reason what is what, such as separating between different entities and different attributes within an entity.

Now, whenever we write reasoning instructions to the LLM, we can just say "Do X based on <article_gudeline> ..." and it will know to reference the information from these particular XML tags.

We chose XML over JSON because it's easier to read and less verbose, which, on average, translates to fewer input tokens while still being easy for humans to read.

### Research Entity

The `Research` entity loads the research file generated by the Nova deep research agent that contains factual data, references, and information that supports the article's claims. It also extracts and validates image URLs from the research file.

From `brown.entities.research`:

```python
import re
from functools import cached_property

from loguru import logger
from pydantic import BaseModel

from brown.entities.mixins import ContextMixin
from brown.utils.a import asyncio_run, run_jobs
from brown.utils.network import is_image_url_valid


class Research(BaseModel, ContextMixin):
    content: str
    max_image_urls: int = 30

    @cached_property
    def image_urls(self) -> list[str]:
        # Extract image URLs using regex
        image_urls = re.findall(
            r"(?!data:image/)https?://[^\s]+\.(?:jpg|jpeg|png|bmp|webp)",
            self.content,
            re.IGNORECASE,
        )
        # Validate URLs asynchronously by pinging them.
        jobs = [is_image_url_valid(url) for url in image_urls]
        results = asyncio_run(run_jobs(jobs))

        urls = [url for url, valid in zip(image_urls, results) if valid]
        if len(urls) > self.max_image_urls:
            logger.warning(f"Found `{len(urls)} > {self.max_image_urls}` image URLs. Trimming to first {self.max_image_urls}.")
            urls = urls[: self.max_image_urls]

        return urls

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    {self.content}
</{self.xml_tag}>
"""

    def __str__(self) -> str:
        return f"Research(len_content={len(self.content)}, len_image_urls={len(self.image_urls)})"
```

**Advanced Features:**

1. **Image URL Extraction**: Automatically finds image URLs in the research content to manipulate them within the LLM context
2. **URL Validation**: Asynchronously validates that image URLs are accessible by pinging them
3. **Caching**: Uses `@cached_property` to avoid re-extracting URLs
4. **Safety Limits**: Caps the number of images to prevent context overflow

The extracted image URLs can be passed to multimodal models (like Gemini) along with text prompts for richer context.

Let's load the research data:


In [13]:
from brown.loaders import MarkdownResearchLoader

research_loader = MarkdownResearchLoader(uri=Path("research.md"))
research = research_loader.load(working_uri=SAMPLE_DIR)

As before let's load just the first 1500 characters to see everything within the Notebook:

In [14]:
cropped_research = research.model_copy()
cropped_research.content = f"{cropped_research.content[:1200]}..."

pretty_print.wrapped(f"{cropped_research.content}", title="Research (First 1200 Chars)")

----------------------------------- Research (First 1200 Chars) -----------------------------------
  # Research

## Code Sources

<details>
<summary>Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md</summary>

# Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md

## Summary
Repository: google-gemini/gemini-cli
File: README.md
Lines: 211

Estimated tokens: 1.6k

## File tree
```Directory structure:
└── README.md

```

## Extracted content
FILE: README.md
# Gemini CLI

 https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml/badge.svg ](https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml)

 ./docs/assets/gemini-screenshot.png 

This repository contains the Gemini CLI, a command-line AI workflow tool that connects to your
tools, understands your code and accelerates your workflows.

With the Gemini CLI you can:

- Query and edit large codebases in and beyond Gemini's 1M token context 

As context:

In [15]:
pretty_print.wrapped(f"{cropped_research.to_context()}", title="Research")

--------------------------------------------- Research ---------------------------------------------
  
<research>
    # Research

## Code Sources

<details>
<summary>Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md</summary>

# Repository analysis for https://github.com/google-gemini/gemini-cli/blob/main/README.md

## Summary
Repository: google-gemini/gemini-cli
File: README.md
Lines: 211

Estimated tokens: 1.6k

## File tree
```Directory structure:
└── README.md

```

## Extracted content
FILE: README.md
# Gemini CLI

 https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml/badge.svg ](https://github.com/google-gemini/gemini-cli/actions/workflows/ci.yml)

 ./docs/assets/gemini-screenshot.png 

This repository contains the Gemini CLI, a command-line AI workflow tool that connects to your
tools, understands your code and accelerates your workflows.

With the Gemini CLI you can:

- Query and edit large codebases in and beyond Gemini's 

As before, we can see how the Research object it's mapped to it's content surrounded by the `<research>...</research>` XML tags making sure that whenever we want to pass this object to an LLM we respect this rule.

As an important gotcha, we don't have to be too rigid about how we translate these objects to XML. For example, here we completely omitted the `<content>` field. As there is just one field, encoding the attribute name is redundant. The most important element is the root `<research>...</research>` XML tag. 

We intentionally implemented both options to highlight this method's flexibility. 

### Article Examples Entity

The `ArticleExamples` entity contains few-shot examples that demonstrate the desired writing style, structure, and quality. These serve as templates for the LLM to understand what kind of output is expected. Intuitively, these can be seen as the on-demand training set that specializes the LLM on our concrete task.

From `brown.entities.articles`:

```python
from pydantic import BaseModel
from brown.entities.mixins import ContextMixin


class ArticleExample(BaseModel, ContextMixin):
    content: str

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    {self.content}
</{self.xml_tag}>
"""

    def __str__(self) -> str:
        return f"ArticleExample(len_content={len(self.content)})"


class ArticleExamples(BaseModel, ContextMixin):
    examples: list[ArticleExample]

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
        {"\n".join([example.to_context() for example in self.examples])}
</{self.xml_tag}>
"""
```

**Composition Pattern:**

- `ArticleExample`: Represents a single example article
- `ArticleExamples`: Contains multiple examples and composes their context representations
- When converted to context, all examples are nested within the parent XML tags

This pattern allows us to provide multiple examples to the LLM, showing consistency across different articles.

Let's load the few-shot examples:


In [16]:
from brown.loaders import MarkdownArticleExampleLoader

examples_loader = MarkdownArticleExampleLoader(uri=EXAMPLES_DIR)
article_examples = examples_loader.load()

cropped_article_examples = article_examples.model_copy()
for example in cropped_article_examples.examples:
    example.content = f"{example.content[:500]}..."

pretty_print.wrapped(f"{article_examples.examples[0]}", title="First Article Example (First 500 Chars)", width=120)

--------------------------------------- First Article Example (First 500 Chars) ---------------------------------------
  ArticleExample(len_content=503)
------------------------------------------------------------------------------------------------------------------------


Let's also call the `to_context()` method on this nested structure:

In [17]:
pretty_print.wrapped(
    f"{cropped_article_examples.to_context()}", title="Article Examples as Context (First 500 Chars)", width=120
)

------------------------------------ Article Examples as Context (First 500 Chars) ------------------------------------
  
<article_examples>
        
<article_example>
    # Lesson 3: Context Engineering

AI applications have evolved rapidly. In 2022, we had simple chatbots for question-answering. By 2023, Retrieval-Augmented Generation (RAG) systems connected LLMs to domain-specific knowledge. 2024 brought us tool-using agents that could perform actions. Now, we are building memory-enabled agents that remember past interactions and build relationships over time.

In our last lesson, we explored how to choose between AI agents and LLM workflows when designing a sy...
</article_example>


<article_example>
    # Lesson 4: Structured Outputs

In our previous lessons, we laid the groundwork for AI Engineering. We explored the AI agent landscape, looked at the difference between rule-based LLM workflows and autonomous AI agents, and covered context engineering: the art of feeding the righ

Note how we represented a list of objects with XML tags, using the following nested structured:
```
<article_examples>
    <article_example>
    ...
    </article_example>
    <article_example>
    ...
    </article_example>
</article_examples>
```
Like this we can represent any list of objects when transforming them from Pydantic entities to LLM context.

### Profiles Entity

Profiles are the secret sauce of the Brown agent. They provide detailed instructions that control different aspects of how we expect the article to look and sound like.

First, let's look at how we structured them as Pydantic classes, and then we will explain in more detail the role of each one.

From `brown.entities.profiles`:

```python
from pydantic import BaseModel
from brown.entities.mixins import ContextMixin


class Profile(BaseModel, ContextMixin):
    name: str
    content: str

    def to_context(self) -> str:
        return f"""
<{self.xml_tag}>
    <name>{self.name}</name>
    <content>{self.content}</content>
</{self.xml_tag}>
"""


class CharacterProfile(Profile):
    pass


class ArticleProfile(Profile):
    pass


class StructureProfile(Profile):
    pass


class MechanicsProfile(Profile):
    pass


class TerminologyProfile(Profile):
    pass


class TonalityProfile(Profile):
    pass


class ArticleProfiles(BaseModel):
    character: CharacterProfile
    article: ArticleProfile
    structure: StructureProfile
    mechanics: MechanicsProfile
    terminology: TerminologyProfile
    tonality: TonalityProfile
```

**Profile Hierarchy:**

1. **Base `Profile` Class**: Provides common structure (name + content) and context conversion
2. **Specialized Profile Classes**: Inherit from `Profile` for specialized context elements
3. **`ArticleProfiles` Container**: Holds all six profile types together

**Why Separate Profile Classes?**

Even though they have the same structure, separate classes provide:
- Type safety (can't accidentally swap profile types)
- Context clarity (each profile will have its own XML tag when added as context)

Let's load all the profiles:


In [18]:
from brown.loaders import MarkdownArticleProfilesLoader

profiles_input = {
    "article": PROFILES_DIR / "article_profile.md",
    "character": PROFILES_DIR / "character_profiles" / "paul_iusztin.md",
    "mechanics": PROFILES_DIR / "mechanics_profile.md",
    "structure": PROFILES_DIR / "structure_profile.md",
    "terminology": PROFILES_DIR / "terminology_profile.md",
    "tonality": PROFILES_DIR / "tonality_profile.md",
}

profiles_loader = MarkdownArticleProfilesLoader(uri=profiles_input)
article_profiles = profiles_loader.load()

profile_sizes = {
    "Character Profile": len(article_profiles.character.content),
    "Article Profile": len(article_profiles.article.content),
    "Structure Profile": len(article_profiles.structure.content),
    "Mechanics Profile": len(article_profiles.mechanics.content),
    "Terminology Profile": len(article_profiles.terminology.content),
    "Tonality Profile": len(article_profiles.tonality.content),
}
profile_sizes["Total"] = sum(profile_sizes.values())
pretty_print.wrapped(
    profile_sizes,
    title="Profile Sizes (in characters)",
)

---------------------------------- Profile Sizes (in characters) ----------------------------------
  {
  "Character Profile": 3033,
  "Article Profile": 13074,
  "Structure Profile": 22660,
  "Mechanics Profile": 4747,
  "Terminology Profile": 10730,
  "Tonality Profile": 4192,
  "Total": 58436
}
----------------------------------------------------------------------------------------------------


Now, let's take a look at the article character profile:

In [19]:
article_profile_copy = article_profiles.article.model_copy()
article_profile_copy.content = f"{article_profile_copy.content[:400]}..."

pretty_print.wrapped(f"{article_profile_copy.content}", title="Article Profile (First 400 Chars)", width=170)

------------------------------------------------------------------- Article Profile (First 400 Chars) -------------------------------------------------------------------
  ## Tonality

You should write in a humanized way as writing a blog article or book.

Write the description of ideas as fluid as possible. Remember that you are writing a book or blog article. Thus, everything should flow naturally, without too many bullet points or subheaders. Use them only when it really makes sense. Otherwise, stick to normal paragraphs.

## General Article Structure

The articl...
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Now, let's call the `to_context()` method. We will show only the first 500 characters to see how the enclosing XML tags logic works:

In [20]:
pretty_print.wrapped(f"{article_profile_copy.to_context()}", title="Article Profile as Context (First 400 Chars)")


--------------------------- Article Profile as Context (First 400 Chars) ---------------------------
  
<article_profile>
    <name>article</name>
    <content>## Tonality

You should write in a humanized way as writing a blog article or book.

Write the description of ideas as fluid as possible. Remember that you are writing a book or blog article. Thus, everything should flow naturally, without too many bullet points or subheaders. Use them only when it really makes sense. Otherwise, stick to normal paragraphs.

## General Article Structure

The articl...</content>
</article_profile>

----------------------------------------------------------------------------------------------------


And if we call the `to_context()` method on a different profile we will see it picked on its own XML tag, reason why it was important to create different classes for each:

In [21]:
structure_profile_copy = article_profiles.structure.model_copy()
structure_profile_copy.content = f"{structure_profile_copy.content[:400]}..."

pretty_print.wrapped(f"{structure_profile_copy.to_context()}", title="Structure Profile as Context (First 400 Chars)")


-------------------------- Structure Profile as Context (First 400 Chars) --------------------------
  
<structure_profile>
    <name>structure</name>
    <content>## Sentence and paragraph length patterns

Write sentences 5–25 words; allow occasional 30-word 'story' sentences. Keep paragraphs ≤ 80 words; allow an occasional 1-sentence paragraph to emphasize a point.

- Good examples:
  - four 18-word sentence, as a paragraph of 72 words.
  - Ocassional 1-sentece paragraph.
- Bad examples:
  - Frequent 40-word run-ons.
  - five 18-word sentence, as a paragra...</content>
</structure_profile>

----------------------------------------------------------------------------------------------------


Note how we represent the `name` and `content` attributes under the `<name>...</name>` and `<content>...</content>` child XML tags, under the `<structure_profile>...</structure_profile>` parent XML tag.

## 5. Understanding the Role of the Profiles

Now that we've loaded all the profiles, let's understand what each one does and why it's essential for generating high-quality content. Profiles are the key to making the Brown writing workflow produce consistent, high-quality output that matches your desired style and voice.

We have four general profiles: mechanics, structure, terminology, and tonality. Then, we have two specific profiles that cover: article and character

The general ones are used to instruct the LLM on general rules of thumb for what a professional piece of content should look like, such as defining the voice, setting sentence length, and avoiding AI slop. At the same time, the specific ones are used to configure the LLM to produce our ideal article and provide a unique kick to the writing process.

As shown in image below, you can replace the examples, article, or character profile with your own to begin generating other article formats and different content types, such as video transcripts or social media posts, and add your own personality to the writing process. 

<img src="https://raw.githubusercontent.com/iusztinpaul/agentic-ai-engineering-course-data/main/images/l22_writing_workflow_profiles.png" alt="Profiles"/>

Keeping things modular in separate Markdown files, even at the context-engineering level, makes it much easier to grow this into a full-fledged, scalable product. For example, if you introduce the concept of a user, each user can configure their own character profile, which is dynamically added to the workflow at the beginning without altering any other aspect of the implementation.

> [!NOTE]
> 💡 As each profile has hundreds of lines, we won’t be able to show them one-on-one within the lesson. We will only show relevant pieces from each, but we encourage you to open each file and skim through it.


Let's go over every 6 profiles:

1. **Mechanics Profile (General)**
    
    This profile governs the technical aspects of writing, encompassing word choice and paragraph structure. It ensures the text is readable, consistent, and grammatically aligned with our standards. 
    
    Some concrete examples:
    
    - **Active vs. passive voice usage:** “Always strive for an active voice.”
    - **Point of View:** “The piece is created by a team writing for a single reader... always use ‘we,’ ‘our,’ and ‘us’ to refer to the team... and ‘you’ or ‘your’ to address the reader.”
    - **Punctuation preferences:** “Avoid semicolons ‘;’ or em-dashes ‘—’ to add phauses... Instead, split it into two sentences.”
    
    See the complete profile at `inputs/profiles/mechanics_profile.md`
    
2. **Structure Profile (General)**
    
    This profile defines how content is visually and logically organized to ensure readability. It governs the hierarchy of headers, the formatting of lists and code, and the placement of media.
    
    Some concrete examples:
    
    - **Sentence and paragraph length patterns:** “Write sentences 5–25 words... Keep paragraphs ≤ 80 words.”
    - **Sections Sub-Heading Formatting:** “Use at maximum H3... sub-headers... avoid using H4/H5/H6 sub-headings at all costs.”
    - **Formatting Media:** “We handle three types of media: Tables... Diagrams... Images... All the media items have a unique identifier... and a small description.”
    - **Formatting Code:** “Avoid describing big chunks of code that go over 35 lines... split the code into logical groups.”
    
    See the complete profile at `inputs/profiles/structure_profile.md`
    
3. **Terminology Profile (General)**
    
    This profile guides word choice to ensure we speak the reader’s language without sounding robotic. It guides the choice of words when transitioning between sentences or paragraphs. Also, it strictly enforces a ban on “AI Slop”, the fluffy, distinctive style often generated by default LLM outputs.
    
    Some concrete examples:
    
    - **Word Choice Patterns:** “Avoid using complex words... Use a casual and direct vocabulary... Use a concrete, hands-on language.”
    - **Descriptive Language Patterns:** “Be excited and personal about positive outcomes... Be realistic, pragmatic, and resilient about negative outcomes.”
    - **AI Slop Banned Expressions List:** Avoid words like “delve”, “paramount”, “thrive”, “realm”, “dive deep into”, “unlock”, “unleash”, “game-changer”.
    
    See the complete profile at `inputs/profiles/terminology_profile.md`
    
4. **Tonality Profile (General)**
    
    This profile sets the emotional resonance and personality of the text. It ensures the writing feels both human and approachable, rather than corporate or purely academic. It guides the level of difficulty and emotion of each word.
    
    Some concrete examples:
    
    - **Primary voice characteristics:** “human, technical, informative, casual, friendly, confident, direct, professional, concise.”
    - **Formality level:** “7/10 - As we write technical professional content, keep it somewhat formal, but NOT too formal.”
    - **On-Brand Tones (Desired):** “Joyful, Excited... Friendly, Sincere... Diplomatic, Empathetic... Direct, Assertive.”
    - **Off-Brand Tones (Undesired):** “Dissatisfied, Dismissive... Disapproving, Accusatory... Overconfident... Informal.”
    
    See the complete profile at `inputs/profiles/tonality_profile.md`
    
5. **Article Profile (Specific)**
    
    This profile contains rules specific to the article format, ensuring a consistent narrative flow. It dictates how to structure the article, transition between sections, and handle citations.
    
    Some concrete examples:
    
    - **General Article Structure:** “The article is a collection of blocks that flow naturally... It starts with one introduction, continues with multiple sections... and wraps up with a conclusion.”
    - **Introduction, Section, Conclusion Guidelines:** “Introduction: short summary... presenting the ‘why’ and ‘what’... Sections: present the ‘how’... Conclusion: very short wrap-up.”
    - **Narrative Flow of the Article:** “Problem -> Why other solutions fail -> Theoretical solution -> Examples -> Advanced theory -> Complex example -> Connection to field.”
    - **Referencing Ideas Between Sections:** “Avoid repeating the same idea twice... You may, however, revisit a prior point from a different perspective.”
    - **References:** “References written in APA 7th edition format.”
    
    You can swap this profile with a LinkedIn post or video transcript profile to create different types of outputs.
    
    See the full profile at `inputs/profiles/article_profile.md`
    
6. **Character Profile (Specific)**
    
    This injects a specific persona into the writing, making it feel authentic and authoritative. For this course, we use our “Paul Iusztin” profile because it felt the most natural for us, as one of the lead instructors. Here, we can add a description of the character, its style, experience, or any other aspect that makes it unique. 
    
    Some concrete examples:
    
    - **About Paul Iusztin:** “Senior AI Engineer... Author of the bestseller LLM Engineer’s Handbook... Founder of the Decoding AI Magazine.”
    - **Similar Personas:** “Andrew Ng, Chip Huyen, Sebastian Raschka, Louis-François Bouchard, Maxime Labonne, Jason Liu, Lex Fridman, Aleksa Gordic.”
    - **Style:** “Real... Trust... Minimalist... Simple... Controversy.”
    
    You can switch this from the Paul Iusztin voice to your own voice, or to another popular character such as Richard Feynman, to inject different personalities and backgrounds.
    
    See the full profile at `inputs/profiles/character_profiles/paul_iusztin.md`
    

In practice, the line between some profiles, such as terminology and tonality, or mechanics and structure, can get blurry. We could merge them into one or keep them as is. We haven’t spent much time optimizing which instruction goes where, and we’ve mostly added them intuitively.

Therefore, we recommend not overthinking the boundaries between profiles, but instead focusing on the specific instructions within them. The most crucial distinction to preserve is between the generic, article, and character profiles. Within the generic profiles, you have the freedom to change this categorization in future iterations.


## 6. Unifying LLM Calls

Before we dive into the workflow nodes, let's understand how the Brown agent manages different LLM APIs and configurations.
### The `get_model` Function

The central function for model initialization is `get_model` from `brown.models.get_model`. It provides a unified interface for creating LLM instances across the codebase.

From `brown.models.get_model`:

```python
import json

from langchain.chat_models import init_chat_model
from langchain_core.language_models import BaseChatModel

from brown.config import get_settings
from brown.models.config import DEFAULT_MODEL_CONFIGS, ModelConfig, SupportedModels
from brown.models.fake_model import FakeModel

MODEL_TO_REQUIRED_API_KEY = {
    SupportedModels.GOOGLE_GEMINI_30_PRO: "GOOGLE_API_KEY",
    SupportedModels.GOOGLE_GEMINI_25_PRO: "GOOGLE_API_KEY",
    SupportedModels.GOOGLE_GEMINI_25_FLASH: "GOOGLE_API_KEY",
    SupportedModels.GOOGLE_GEMINI_25_FLASH_LITE: "GOOGLE_API_KEY",
}


def get_model(model: SupportedModels, config: ModelConfig | None = None) -> BaseChatModel:
    if model == SupportedModels.FAKE_MODEL:
        if config and config.mocked_response is not None:
            if hasattr(config.mocked_response, "model_dump"):
                mocked_response_json = config.mocked_response.model_dump(mode="json")
            else:
                mocked_response_json = json.dumps(config.mocked_response)
            return FakeModel(responses=[mocked_response_json])
        else:
            return FakeModel(responses=[])

    config = config or DEFAULT_MODEL_CONFIGS.get(model) or ModelConfig()
    model_kwargs = {
        "model": model.value,
        **config.model_dump(),
    }

    required_api_key = MODEL_TO_REQUIRED_API_KEY.get(model)
    if required_api_key:
        settings = get_settings()
        if not getattr(settings, required_api_key):
            raise ValueError(f"Required environment variable `{required_api_key}` is not set")
        else:
            model_kwargs["api_key"] = getattr(settings, required_api_key)

    return init_chat_model(**model_kwargs)
```

**Key Features:**

1. **API Key Management**: Automatically pulls credentials from settings
2. **Default Configurations**: Falls back to sensible defaults
3. **LangChain Integration**: Uses `init_chat_model` for consistent interface between different LLMs


### Model Configuration Structures

The system uses three key structures for model configuration:

From `brown.models.config`:

```python
from enum import StrEnum
from typing import Any

from pydantic import BaseModel, Field


class SupportedModels(StrEnum):
    GOOGLE_GEMINI_30_PRO = "google_genai:gemini-3-pro-preview"
    GOOGLE_GEMINI_25_PRO = "google_genai:gemini-2.5-pro"
    GOOGLE_GEMINI_25_FLASH = "google_genai:gemini-2.5-flash"
    GOOGLE_GEMINI_25_FLASH_LITE = "google_genai:gemini-2.5-flash-lite"
    FAKE_MODEL = "fake"


class ModelConfig(BaseModel):
    temperature: float = 0.7
    top_k: int | None = None
    n: int = 1
    response_modalities: list[str] | None = None
    include_thoughts: bool = False
    thinking_budget: int | None = Field(
        default=None,
        ge=0,
        description="If reasoning is available, the maximum number of tokens the model can use for thinking.",
    )
    max_output_tokens: int | None = None
    max_retries: int = 6

    mocked_response: Any | None = None


DEFAULT_MODEL_CONFIGS = {
    "google_genai:gemini-2.5-pro": ModelConfig(
        temperature=0.7,
        include_thoughts=False,
        thinking_budget=1000,
        max_retries=3,
    ),
    "google_genai:gemini-2.5-flash": ModelConfig(
        temperature=1,
        thinking_budget=1000,
        include_thoughts=False,
        max_retries=3,
    ),
    "google_genai:gemini-2.0-flash-exp": ModelConfig(
        temperature=0.7,
        thinking_budget=1000,
        include_thoughts=False,
        max_retries=3,
    ),
}
```

**Design Insights:**

1. **`SupportedModels` Enum**: Type-safe model selection
2. **`ModelConfig` Pydantic Model**: Validates configuration parameters
3. **`DEFAULT_MODEL_CONFIGS` Dict**: Pre-configured settings for each model

This allows different nodes in the workflow to use different models with different configurations, optimizing for specific tasks (e.g., using faster models for media generation, more powerful models for article writing).


### Usage Throughout the Codebase

Every node in the workflow uses `get_model` to instantiate its LLM. This provides:

- **Consistency**: Same interface everywhere
- **Flexibility**: Easy to swap models for different nodes
- **Configuration**: Centralized model settings
- **Testing**: Can use FakeModel for tests

Let's try it out with a simple example:


In [22]:
from brown.models import ModelConfig, SupportedModels, get_model

# Create a model instance with custom configuration
model_config = ModelConfig(temperature=0.5, max_output_tokens=100)
model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH, config=model_config)

# Test it with a simple prompt
response = await model.ainvoke([{"role": "user", "content": "Say hello in one sentence!"}])
print(f"Model response: {response.content}")

Model response: 


Let's see how the fake model works as well:

In [23]:
model_config = ModelConfig(temperature=0.5, max_output_tokens=100, mocked_response="This is a fake response!")
model = get_model(SupportedModels.FAKE_MODEL, config=model_config)

# Test it with a simple prompt
response = await model.ainvoke([{"role": "user", "content": "Say hello in one sentence!"}])
print(f"Model response: {response.content}")

Model response: "This is a fake response!"


## 7. Generating Media Items Using the Orchestrator-Worker Pattern

Now let's explore one of the most interesting architectural patterns in the Brown agent: the orchestrator-worker pattern for media generation. This pattern efficiently delegates specialized tasks to expert workers.

### Understanding Node Abstractions

First, let's understand the base abstractions that all workflow nodes inherit from.

From `brown.nodes.base`:

```python
from abc import ABC, abstractmethod
from typing import Any, Iterable, Literal, TypedDict

from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool


class ToolCall(TypedDict):
    name: str
    args: dict[str, Any]
    id: str
    type: Literal["tool_call"]


class Toolkit(ABC):
    """Base class for toolkits following LangChain's toolkit pattern."""

    def __init__(self, tools: list[BaseTool]) -> None:
        self._tools: list[BaseTool] = tools
        self._tools_mapping: dict[str, BaseTool] = {tool.name: tool for tool in self._tools}

    def get_tools(self) -> list[BaseTool]:
        """Get all registered media item generation tools."""
        return self._tools.copy()

    def get_tools_mapping(self) -> dict[str, BaseTool]:
        """Get a mapping of tool names to tool instances."""
        return self._tools_mapping

    def get_tool_by_name(self, name: str) -> BaseTool | None:
        """Get a specific tool by name."""
        return self._tools_mapping.get(name)


class Node(ABC):
    def __init__(self, model: Runnable, toolkit: Toolkit) -> None:
        self.toolkit = toolkit
        self.model = self._extend_model(model)

    def _extend_model(self, model: Runnable) -> Runnable:
        # Can be overridden to bind tools, structured output, etc.
        return model

    def build_user_input_content(self, inputs: Iterable[str], image_urls: list[str] | None = None) -> list[dict[str, Any]]:
        """Build multimodal input content with optional images."""
        messages: list[dict[str, Any]] = []
        if image_urls:
            for image_url in image_urls:
                messages.append({"type": "image_url", "image_url": {"url": image_url}})
        
        for input_ in inputs:
            messages.append({"type": "text", "text": input_})

        return messages

    @abstractmethod
    async def ainvoke(self) -> Any:
        pass
```

**Key Abstractions:**

1. **`ToolCall`**: TypedDict representing a tool call (name, args, id)
2. **`Toolkit`**: Manages a collection of tools that can be passed to a node
3. **`Node`**: Base class for all workflow nodes
   - Takes a model and toolkit
   - Can extend the model (bind tools, structured output)
   - Supports multimodal input (text + images)
   - Abstract `ainvoke` method for execution


### The MediaGeneratorOrchestrator Node

The orchestrator analyzes the article guideline and research to identify what media items need generation, then delegates to specialized worker tools.

Key implementation details from `brown.nodes.media_generator.MediaGeneratorOrchestrator`:

**1. Class Initialization:**
```python
class MediaGeneratorOrchestrator(Node):
    system_prompt_template = "..."

    def __init__(
        self,
        article_guideline: ArticleGuideline,
        research: Research,
        model: Runnable,
        toolkit: Toolkit,
    ) -> None:
        self.article_guideline = article_guideline
        self.research = research
        super().__init__(model, toolkit)
```

**2. Model Extension (Tool Binding):**
```python
def _extend_model(self, model: Runnable) -> Runnable:
    model = cast(BaseChatModel, super()._extend_model(model))
    model = model.bind_tools(self.toolkit.get_tools(), tool_choice="any")
    return model
```

The orchestrator binds all available worker tools to the model, allowing it to call multiple tools.

💡 Through tools we can easily extend the orchestrator with multiple worker types without even touching the orchestrator, making this a fully modular implementation.

**3. Async Invocation:**
```python
async def ainvoke(self) -> list[ToolCall]:
    system_prompt = self.system_prompt_template.format(
        article_guideline=self.article_guideline.to_context(),
        research=self.research.to_context(),
    )
    user_input_content = self.build_user_input_content(
        inputs=[system_prompt], 
        image_urls=self.research.image_urls
    )
    inputs = [{"role": "user", "content": user_input_content}]
    response = await self.model.ainvoke(inputs)
    
    if isinstance(response, AIMessage) and response.tool_calls:
        jobs = cast(list[ToolCall], response.tool_calls)
    else:
        jobs = []
    
    return jobs
```

Returns a list of `ToolCall` objects describing what media items to generate.

**4. System Prompt:**

The orchestrator uses a detailed system prompt that instructs it to:
- Analyze the article guideline for media requirements
- Look for explicit indicators like "Render a Mermaid diagram"
- Call appropriate worker tools with detailed descriptions
- Handle cases where no media is needed

Here is the full system prompt attached to the node:
```python
class MediaGeneratorOrchestrator(Node):
    system_prompt_template = """You are an Media Generation Orchestrator responsible for analyzing article 
guidelines and research to identify what media items need to be generated for the article.

Your task is to:
1. Carefully analyze the article guideline and research content provided
2. Identify ALL explicit requests for media items (diagrams, charts, visual illustrations, etc.)
3. For each identified media requirement, call the appropriate tool to generate the media item
4. Provide clear, detailed descriptions for each media item based on the guideline requirements and research context

## Analysis Guidelines

**Look for these explicit indicators in the article guideline:**
- Direct mentions: "Render a Mermaid diagram", "Draw a diagram", "Create a visual", "Illustrate with", etc.
- Visual requirements: "diagram visually explaining", "chart showing", "figure depicting", "visual representation"
- Process flows: descriptions of workflows, architectures, data flows, or system interactions
- Structural elements: hierarchies, relationships, comparisons, or step-by-step processes

**Key places to look:**
- Section requirements and descriptions  
- Specific instructions mentioning visual elements
- Complex concepts that would benefit from visual explanation
- Architecture or system descriptions
- Process flows or workflows

## Tool Usage Instructions

You will call multiple tools to generate the media items. You will use the tool that is most appropriate for the media item you are 
generating.

For each identified media requirement:

**When to use MermaidDiagramGenerator:**
- Explicit requests for Mermaid diagrams
- System architectures and workflows
- Process flows and data pipelines  
- Organizational structures or hierarchies
- Flowcharts for decision-making processes
- Sequence diagrams for interactions
- Entity-relationship diagrams
- Class diagrams for software structures
- State diagrams for system states
- Mind maps for concept relationships

**Description Requirements:**
When calling tools, provide detailed descriptions that include:
- The specific purpose and context from the article guideline
- Key components that should be included based on the research
- The type of diagram most appropriate (flowchart, sequence, architecture, etc.)
- Specific elements, relationships, or flows to highlight
- Any terminology or technical details from the research

## Example Analysis Process

1. **Scan the guideline** for phrases like:
   - "Render a Mermaid diagram of..."
   - "Draw a diagram showing..."
   - "Illustrate the architecture..."
   - "Visual representation of..."

2. **For each found requirement:**
   - Extract the specific context and purpose
   - Identify what should be visualized
   - Determine the most appropriate diagram type
   - Craft a detailed description incorporating research insights

3. **Call the appropriate tool** with the comprehensive description

## Input Context

Here is the article guideline:
{article_guideline}

Here is the research:
{research}

## Your Response

Analyze the provided article guideline and research, then call the appropriate tools for each 
identified media item requirement. Each tool call should include a detailed description that ensures 
the generated media item will be relevant, accurate, and valuable for the article's educational goals.

If no explicit media requirements are found in the guideline, respond with: 
"No explicit media item requirements found in the article guideline."
"""
```

💡 The only element from the orchestrator that is aware about its tools, such as the `MermaidDiagramGenerator`, is the system prompt. Here we did this to ensure it always picks up on what we want, but you can further experiment to make the system prompt more general and choose the right tools solely based on their tool description and interface. Ultimately, doing so, you make the orchestrator fully modular. But sometimes hardcoding stuff in the system prompt to get the job done is totally fine. You just have to be aware about the rules to know when to break them. 

### The ToolNode Abstraction

Worker tools inherit from `ToolNode`, a special type of `Node` that can be converted into a LangChain tool.

From `brown.nodes.base`:

```python
class ToolNode(Node):
    def __init__(self, model: Runnable) -> None:
        super().__init__(model, toolkit=Toolkit(tools=[]))

    def as_tool(self) -> StructuredTool:
        return StructuredTool.from_function(
            coroutine=self.ainvoke,
            name=f"{camel_to_snake(self.__class__.__name__)}_tool",
        )

    @abstractmethod
    async def ainvoke(self) -> Any:
        pass
```

**Key Features:**

- Inherits from `Node` but has no tools itself (empty toolkit)
- `as_tool()` method converts the node into a LangChain `StructuredTool`
- The tool name is automatically derived from the class name
- The tool's coroutine is the node's `ainvoke` method

This allows any `ToolNode` to be easily integrated as a tool into any node's toolkit. Like this we can easily transform each node into a tool that can be passed to another node, providing full composability between nodes.

In our use case we will create a `MediaGeneratorOrchestrator` node that we will transform into a tool that will be passed as a worker to the orchestrator. Let's see how that works.

### The MermaidDiagramGenerator Worker

Before running the `MediaGeneratorOrchestrator`, let's examine a concrete worker implementation from `brown.nodes.tool_nodes.MermaidDiagramGenerator`.

💭 Even if an LLM is capable of generating both the Mermaid diagram and the article at the same time, having a specialized worker that is carefully prompted for generating Mermaid diagrams gives us more control as we can customize how the Mermaid diagrams should look like, carefully prompt engineering special use cases such as treating special characters and guiding the LLM what we expect from the diagram. Ultimately ending up with richer, more beautiful, and working Mermaid diagrams.

**1. Structured Output Model:**
```python
class GeneratedMermaidDiagram(BaseModel):
    content: str = Field(description="The Mermaid diagram code formatted in Markdown format as: ```mermaid\n[diagram content here]\n```")
    caption: str = Field(description="The caption, as a short description of the diagram.")
```

**2. Class Structure:**
```python
class MermaidDiagramGenerator(ToolNode):
    prompt_template = "..."

    def _extend_model(self, model: Runnable) -> Runnable:
        model = cast(BaseChatModel, super()._extend_model(model))
        model = model.with_structured_output(GeneratedMermaidDiagram, include_raw=False)
        return model

    async def ainvoke(self, description_of_the_diagram: str, section_title: str) -> MermaidDiagram:
        """Specialized tool to generate a mermaid diagram from a text description. This tool uses a specialized LLM to
        convert a natural language description into a mermaid diagram.

        Use this tool when you need to generate a mermaid diagram to explain a concept. Don't confuse mermaid diagrams,
        or diagrams in general, with media data, such as images, videos, audio, etc. Diagrams are rendered dynamically
        customized for each article, while media data are static data added as URLs from external sources.
        This tool is used explicitly to dynamically generate diagrams, not to add media data.

        Args:
            description_of_the_diagram: Natural language description of the diagram to generate.
            section_title: Title of the section that the diagram is for.

        Returns:
            The generated mermaid diagram code in Markdown format.

        Raises:
            Exception: If diagram generation fails.

        Examples:
        >>> description = "A flowchart showing data flowing from user input to database"
        >>> diagram = await generate_mermaid_diagram(description)
        >>> print(diagram)
        ```mermaid
        graph LR
            A[User Input] --> B[Processing]
            B --> C[(Database)]
        ```
        """

        try:
            response = await self.model.ainvoke(
                [
                    {
                        "role": "user",
                        "content": self.prompt_template.format(
                            description_of_the_diagram=description_of_the_diagram,
                        ),
                    }
                ]
            )

        except Exception as e:
            logger.exception(f"Failed to generate Mermaid diagram: {e}")

            return MermaidDiagram(
                location=section_title,
                content=f'```mermaid\ngraph TD\n    A["Error: Failed to generate diagram"]\n    A --> B["{str(e)}"]\n```',
                caption=f"Error: Failed to generate diagram: {str(e)}",
            )

        if not isinstance(response, GeneratedMermaidDiagram):
            raise InvalidOutputTypeException(GeneratedMermaidDiagram, type(response))

        return MermaidDiagram(
            location=section_title,
            content=response.content,
            caption=response.caption,
        )
```

**Key Features:**

- Uses structured output to ensure valid diagram generation
- The ainvoke() method has a carefully designed pydoc and signature that will be used when the node is transformed into a tool used by the orchestrator.
- In case of error, we return a placeholder diagram to avoid failing the whole workflow because of a tool failure. Remember that we can run dozens of tools in parallel, thus the change of failure due to things such as rate limits is high.

The prompt template contains detailed instructions for generating valid Mermaid diagrams, including:
- Syntax rules (always use double quotes, never use semicolons)
- Examples of different diagram types (flowcharts, process flows)
- Common mistakes to avoid

Here is the full prompt template:
```python
class MermaidDiagramGenerator(ToolNode):
    prompt_template = """
You are a specialized agent that creates clean, readable Mermaid diagrams from text descriptions.

## Task
Generate a valid Mermaid diagram based on this description:
<description_of_the_diagram>
{description_of_the_diagram}
</description_of_the_diagram>

## Output Format
Return ONLY the Mermaid code block in this exact format:
``mermaid
[diagram content here]
``

## Diagram Types & Examples

### Process Flow
``mermaid
graph LR
    A["Input"] --> B["Process"] --> C["Output"]
    B --> D["Validation"]
    D -->|"Valid"| C
    D -->|"Invalid"| A
``mermaid


### Flowcharts (Most Common)
``mermaid
graph TD
    A["Start"] --> B{{"Decision"}}
    B -->|"Yes"| C["Action 1"]
    B -->|"No"| D["Action 2"]
    C --> E["End"]
    D --> E
``mermaid

... # More examples

## Syntax Rules
1. **Node Labels**: Use square brackets `[Label]` for rectangular nodes
2. **Decisions**: Use curly braces `{{Decision}}` for diamond shapes  
3. **Databases**: Use `[(Label)]` for cylindrical database shapes
4. **Circles**: Use `((Label))` for circular nodes
5. **Arrows**: Use `-->` for solid arrows, `-.->` for dotted arrows
6. **Labels on Arrows**: Use `-->|Label|` for labeled connections
7. **Subgraphs**: Use `subgraph "Title"` and `end` to group elements
8. **Comments**: Use `%%` for comments
9. **ERD Entities**: Use `ENTITY_NAME {{ field_type field_name }}` format
10. **ERD Relationships**: Use `||--o{{`, `||--||`, `}}o--||` for different cardinalities
11. **Class Definitions**: Use `class ClassName {{ +type attribute +method() }}` format
12. **Class Relationships**: Use `<|--` (inheritance), `-->` (association), `--*` (composition)

## Formatting Rules
**ALWAYS wrap node labels in double quotes `"..."` to prevent parsing errors:**

**WRONG** (causes parse errors):
h

**Key formatting requirements:**
- **Always use double quotes** around ALL node labels: `A["Label"]`
- **Never use semicolons** at the end of lines (they're optional and can cause issues)
- **Quote any label** containing: parentheses `()`, commas `,`, periods `.`, colons `:`, or spaces
- **Quote subgraph titles** as well: `subgraph "Title"`

## Styling Guidelines
- **Use default colors only** - do not add color specifications or custom styling
- Do not use `fill:`, `stroke:`, `color:` or any CSS styling properties
- Keep diagrams clean and professional with standard Mermaid appearance
- Focus on structure and clarity, not visual customization

## Key Guidelines
- Keep node labels concise (avoid parentheses and special characters in labels)
- Use clear, logical flow from top to bottom or left to right
- Keep the diagram simple and easy to understand
- Group related elements with subgraphs when helpful
- Maintain consistent spacing and formatting
- Choose the appropriate diagram type for the concept being illustrated

## Common Mistakes to Avoid
- **NEVER use unquoted labels** - always wrap in double quotes: `A["Label"]`
- **NEVER use semicolons** at the end of lines (causes parsing issues)
- **NEVER put parentheses `()` in unquoted labels** - parser treats them as shape tokens
- Don't create overly complex diagrams with too many connections
- Avoid extremely long labels that break the visual flow
- **Never use custom colors or styling** - stick to Mermaid's default appearance

Generate a clean, professional diagram that clearly illustrates the described concept using only default Mermaid 
styling. Remember: ALWAYS use double quotes around ALL labels and NEVER use semicolons.
"""
```


### Example: Generating Media Items with the Orchestrator

Let's see the orchestrator-worker pattern in action by generating multiple Mermaid diagrams based on our article guideline:


In [24]:
import asyncio

from brown.entities.media_items import MediaItem
from brown.models import SupportedModels, get_model
from brown.nodes import MediaGeneratorOrchestrator, MermaidDiagramGenerator, Toolkit

# Create worker tool
diagram_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
mermaid_generator = MermaidDiagramGenerator(model=diagram_model)
toolkit = Toolkit(tools=[mermaid_generator.as_tool()])

# Create orchestrator
orchestrator_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
orchestrator = MediaGeneratorOrchestrator(
    article_guideline=article_guideline,
    research=research,
    model=orchestrator_model,
    toolkit=toolkit,
)

# Get media generation jobs
pretty_print.wrapped("Analyzing article guideline for media requirements...")
media_jobs = await orchestrator.ainvoke()


pretty_print.wrapped("Found {len(media_jobs)} media items to generate")
media_jobs_dict = {}
for i, job in enumerate(media_jobs):
    pretty_print.wrapped(
        {
            "Tool": job["name"],
            "Description": job["args"].get("description_of_the_diagram", "N/A")[:100] + "...",
            "Section": job["args"].get("section_title", "N/A"),
        },
        title=f"Job {i + 1}",
    )

pretty_print.wrapped("Generating media items in parallel...")
coroutines = []
for job in media_jobs:
    tool = orchestrator.toolkit.get_tool_by_name(job["name"])
    if tool:
        coroutines.append(tool.ainvoke(job["args"]))

media_items: list[MediaItem] = await asyncio.gather(*coroutines)

pretty_print.wrapped(f"Generated {len(media_items)} media items successfully!")

----------------------------------------------------------------------------------------------------
  Analyzing article guideline for media requirements...
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
  Found {len(media_jobs)} media items to generate
----------------------------------------------------------------------------------------------------
---------------------------------------------- Job 1 ----------------------------------------------
  {
  "Tool": "mermaid_diagram_generator_tool",
  "Description": "A flowchart illustrating a typical LLM workflow. It should start with an input, show an LLM call or ...",
  "Section": "Understanding the Spectrum: From Workflows to Agents"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- 

Let's take a look at the generated diagrams:

In [25]:
for media_item in media_items:
    pretty_print.wrapped(
        {
            "Caption": media_item.caption,
            "Diagram Code": media_item.content,
        },
        title=media_item.location,
    )

----------------------- Understanding the Spectrum: From Workflows to Agents -----------------------
  {
  "Caption": "A flowchart illustrating a deterministic LLM workflow with tool-calling capabilities.",
  "Diagram Code": "```mermaid\ngraph TD\n    A[\"_Start_\"] --> B[\"User Input\"]\n    B --> C[\"Tool-Calling LLM (Deterministic Tool Selection)\"]\n    C --> D[\"Tool 1: Data Retrieval (Predefined Step)\"]\n    D --> E[\"Tool 2: API Call (Predefined Step)\"]\n    E --> F[\"Tool 3: Data Processing (Predefined Step)\"]\n    F --> G[\"Final Output\"]\n    G --> H[\"_End_\"]\n```"
}
----------------------------------------------------------------------------------------------------
----------------------- Understanding the Spectrum: From Workflows to Agents -----------------------
  {
  "Caption": "A flowchart illustrating the core architecture and dynamic decision-making process of an AI agent, showing its interaction with memory, planning, tools, and internal loops to achieve a goal.

Lastly, let's wrap up the media items into a Pydantic entity:

In [26]:
from brown.entities.media_items import MediaItems

media_items = MediaItems.build(media_items=media_items)

💡 Now, with these classes in place, we can easily plug in new media generators as tools, such as a tool that generates images, one that generates videos, or any other super specialized tool that generates brand assets as desired.

### How do we model multimodal inputs?

Another interesting thing to look at before we dig into the `ArticleWriter` node is how we input multimodal prompts, such as all the images from the research.

We do that through the `build_user_input_content()` method, which is part of the `Node` base class. We usually combine it with the `image_url` extracted from the research as follows:

In [39]:
orchestrator.build_user_input_content(
    inputs=["some random inputs", "some other random inputs"],
    image_urls=orchestrator.research.image_urls[:5]
)

[{'type': 'text', 'text': 'Use the following images as <research> context:'},
 {'type': 'image_url',
  'image_url': {'url': 'https://towardsdatascience.com/wp-content/uploads/2025/06/agent-vs-workflow.jpeg'}},
 {'type': 'text', 'text': 'Use the following images as <research> context:'},
 {'type': 'image_url',
  'image_url': {'url': 'https://contributor.insightmediagroup.io/wp-content/uploads/2025/06/when-agents-win-683x1024.jpeg'}},
 {'type': 'text', 'text': 'Use the following images as <research> context:'},
 {'type': 'image_url',
  'image_url': {'url': 'https://contributor.insightmediagroup.io/wp-content/uploads/2025/06/when-workflows-win-683x1024.jpeg'}},
 {'type': 'text', 'text': 'Use the following images as <research> context:'},
 {'type': 'image_url',
  'image_url': {'url': 'https://contributor.insightmediagroup.io/wp-content/uploads/2025/06/image-116.png'}},
 {'type': 'text', 'text': 'Use the following images as <research> context:'},
 {'type': 'image_url',
  'image_url': {'url'

We pack everything into a dictionary, passing the images as URLs extracted from the research, and add the remaining inputs at the end. Also, note that each URL includes a piece of text that explicitly marks it as <research>. Because we model the context as XML tags, it's easy to use them as unique reference IDs throughout the context window.

We explicitly added the input messages at the bottom. As we don't know how many images we will have as input, we avoid the needle-in-the-haystack problem by always keeping the text at the bottom, where the LLM can focus.

This list of dictionaries will then be passed as a single user input as follows:
```python
user_input_content = self.build_user_input_content(
 inputs=[system_prompt], 
 image_urls=self.research.image_urls
 )
 inputs = [{"role": "user", "content": user_input_content}]
 response = await self.model.ainvoke(inputs)
```

Using this strategy, we can easily extend the multimodal input to include documents, videos, or audio data, as long as they can be encoded as base64, URLs, or binary (as we've seen in the multimodal lesson).

## 8. Generating the Article: Bringing It All Together

The `ArticleWriter` is the heart of the content generation system. It takes all the context we've prepared and generates a high-quality article following all the profile rules.

### Class Structure

From `brown.nodes.article_writer.ArticleWriter`:

**1. Initialization:**
```python
class ArticleWriter(Node):
    system_prompt_template = """..."""
    
    def __init__(
        self,
        article_guideline: ArticleGuideline,
        research: Research,
        article_profiles: ArticleProfiles,
        media_items: MediaItems,
        article_examples: ArticleExamples,
        model: Runnable,
    ) -> None:
        super().__init__(model, toolkit=Toolkit(tools=[]))
        
        self.article_guideline = article_guideline
        self.research = research
        self.article_profiles = article_profiles
        self.media_items = media_items
        self.article_examples = article_examples
```


### The ainvoke Method

**2. Article Generation Logic:**
```python
async def ainvoke(self) -> Article | SelectedText:
    # Format the system prompt with all context
    system_prompt = self.system_prompt_template.format(
        article_guideline=self.article_guideline.to_context(),
        research=self.research.to_context(),
        article_profile=self.article_profiles.article.to_context(),
        character_profile=self.article_profiles.character.to_context(),
        mechanics_profile=self.article_profiles.mechanics.to_context(),
        structure_profile=self.article_profiles.structure.to_context(),
        terminology_profile=self.article_profiles.terminology.to_context(),
        tonality_profile=self.article_profiles.tonality.to_context(),
        media_items=self.media_items.to_context(),
        article_examples=self.article_examples.to_context(),
    )
    
    # Build multimodal input (text + images from research)
    user_input_content = self.build_user_input_content(
        inputs=[system_prompt], 
        image_urls=self.research.image_urls
    )
    
    inputs = [{"role": "user", "content": user_input_content}]
    
    # Generate the article
    written_output = await self.model.ainvoke(inputs)
    if not isinstance(written_output, AIMessage):
        raise InvalidOutputTypeException(AIMessage, type(written_output))
    written_output = cast(str, written_output.text)
    
    return Article(content=written_output)
```

The method:
1. Combines all context using `to_context()` methods
2. Supports multimodal input with research images
3. Check if we get the expected output. As the article generation is a critical step within our workflow, we want to fail the whole workflow if this node fails.
4. Returns an `Article` entity


### The System Prompt (Key Sections)

The `ArticleWriter` uses an extensive system prompt that includes all the context. Here are the key sections (simplified for clarity):

**3. System Prompt Structure:**
```python
class ArticleWriter(Node):
    system_prompt_template = """
You are Brown, a professional human writer specialized in writing technical, educative and informational articles
about AI. 

Your task is to write a high-quality article, while providing you the following context:
- **article guideline:** the user intent describing how the article should look like. Specific to this particular article.
- **research:** the factual data used to support the ideas from the article guideline. Specific to this particular article.
- **article profile:** rules specific to writing articles. Generic for all articles.
- **character profile:** the character you will impersonate while writing. Generic for all content.
- **structure profile:** Structure rules guiding the final output format. Generic for all content.
- **mechanics profile:** Mechanics rules guiding the writing process. Generic for all content.
- **terminology profile:** Terminology rules guiding word choice and phrasing. Generic for all content.
- **tonality profile:** Tonality rules guiding the writing style. Generic for all content.

Each of these will be carefully considered to guide your writing process. You will never ignore or deviate from these rules. These
rules are your north star, your bible, the only reality you know and operate on. They are the only truth you have.

## Character Profile

To make the writing more personable, you will impersonate the following character profile. The character profile 
will anchor your identity and specify things such as your:
- **personal details:** name, age, location, etc.
- **working details:** company, job title, etc.
- **artistic preferences:** it's niche, core content pillars, style, tone, voice, etc.

What to avoid using the character profile for:
- explicitly mentioning the character profile in the article, such as "I'm Paul Iusztin, founder of Decoding AI." Use
it only to impersonate the character and make the writing more personable. For example if you are "Paul Iusztin",
you will never say all the time "I'm Paul Iusztin, founder of Decoding AI." as people already know who you are.
- using the character profile to generate article sections, such as "Okay, I'm Paul Iusztin, founder of Decoding AI. 
Let's cut through the hype and talk real engineering for AI agents." Use the character profile only to adapt the
writing style and introduce references to the character. Nothing more.

Here is the character profile:
{character_profile}

## Research

When using factual data to write the article, anchor your results exclusively in information from the given 
<research> or <article_guideline> tags. Avoid, at all costs, using factual information from your internal knowledge.

The <research> will contain most of the factual data to write the article. But the user might add additional information
within the <article_guideline>. 

Thus, always prioritize the factual data from the <article_guideline> over the <research>.

Here is the research you will use as factual data for writing the article:
{research}

## Article Examples

Here is a set of article examples you will use to understand how to write the article:
{article_examples}

## Tonality Profile

Here is the tonality profile, describing the tone, voice and style of the writing:
{tonality_profile}

## Terminology Profile

Here is the terminology profile, describing how to choose the right words and phrases
to the target audience:
{terminology_profile}

## Mechanics Profile

Here is the mechanics profile, describing how the sentences and words should be written:
{mechanics_profile}

## Structure Profile

Here is the structure profile, describing general rules on how to structure text, such as the sections, paragraphs, lists,
code blocks, or media items:
{structure_profile}

## Media Items

Within the <article_guideline>, the user requested to include all types of media items, such as tables, diagrams, images, etc. Some of the 
media items will be present inside the <research> or <article_guideline> tags as links. But often, we will have to generate the 
media items ourselves.

Thus, here is the list of media items that we already generated before writing the article that should be included as they are:
{media_items}

The list contains the <location> of each media item to know where to place it within the article. The location is the section title, 
inferred from the <article_guideline> outline. Based on the <location>, locate the generated media item within the <article_guideline>, 
and use it as is when writing the article.

Replace the media item requirements from the <article_guideline> with the generated media item and its caption. We always
want to group a media item with its caption.

## Article Profile

Here is the article profile, describing particularities on how the end-to-end article should look like:
{article_profile}

## Article Guideline: 

Here is the article guideline, representing the user intent, describing how the actual article should look like:
{article_guideline}

You will always start understand what to write by reading the <article_guideline>.

As the <article_guideline> represents the user intent, it will always have priority over anything else. If any information
contradicts between the <article_guideline> and other rules, you will always pick the one from the <article_guideline>.

Avoid using the whole <research> when writing the article. Extract from the <research> only what is useful to respect the 
user intent from the <article_guideline>. Still, always anchor your content based on the facts from the <research> or <article_guideline>.

Always prioritize the facts directly passed by the user in the <article_guideline> over the facts from the <research>. Avoid at all costs 
to use your internal knowledge when writing the article.

The <article_guideline> will ALWAYS contain:
- all the sections of the article expected to be written, in the correct order
- a level of detail for each section, describing what each section should contain. Depending on how much detail you have in a
particular section of the <article_guideline>, you will use more or less information from the <research> tags to write the section.

The <article_guideline> can ALSO contain:
- length constraints for each section, such as the number of characters, words or reading time. If present, you will respect them.
- important (golden) references as URLs or titles present in the <research> tags. If present, always prioritize them over anything else 
from the <research>.
- information about anchoring the article into a series such as a course or a book. Extremely important when the article is part of 
something bigger and we have to anchor the article into the learning journey of the reader. For example, when introducing concepts
in previous articles that we don't want to reintroduce into the current one.
- concrete information about writing the article. If present, you will ALWAYS prioritize the instructions from the <article_guideline> 
over any other instructions.

## Article Outline

Internally, based on the <article_guideline>, before starting to write the article, you will plan an article outline, 
as a short summary of the article, describing what each section contains and in what order.

Here are the rules you will use to generate the article outline:
- The user's <article_guideline> always has priority! If the user already provides an article outline or a list of sections, 
you will use them instead of generating a new one.
- If the section titles are already provided in the <article_guideline>, you will use them as is, with 0 modifications.
- Extract the core ideas from the <article_guideline> and lay them down into sections.
- Your internal description of each section will be verbose enough for you to understand what each section contains.
- Ultimately, the CORE scope of the article outline is to have an internal process that verifies that each section is anchored into the
<article_guideline>, <research> and all the other profiles.
- Before starting writing the final article, verify that the flow of ideas between the sections, from top to bottom, 
is coherent and natural.

## Chain of Thought

1. Plan the article outline
2. Write the article following the article outline and all the other constraints.
3. Check if all the constraints are respected. Edit the article if not.
4. Return ONLY the final version of the article.

With that in mind, based on the <article_guideline>, you will write an in-depth and high-quality article following all 
the <research>, guidelines and profiles.
"""
```

**Key Prompt Engineering Techniques:**

1. **Clear Role Definition**: "You are Brown, a professional human writer..."
2. **Structured Context**: Each piece of context has its own section that is automatically enclosed by XML tags due to the `ContextMixin.to_context()` interface. 
3. **Priority Guidance**: The "Article guideline" which is the user input, always has priority over everything else.
4. **Chain of Thought**: Explicit reasoning steps
5. **Constraint Adherence**: Multiple reminders to follow all profiles to ensure the LLM picks up on our core instructions.
6. **Grounding in Facts**: "Anchor exclusively in research, avoid internal knowledge" to avoid hallucinations

This comprehensive prompt ensures the LLM has all the information and instructions it needs to generate high-quality, consistent content.


## 9. Running the Writing Workflow End-To-End

Now let's put everything together and run the complete workflow to generate an article using our test data. 

This time, we will use a more comprehensive test sample containing a more detailed `article_guideline.md` and more facts within the `research.md` file to show what a professional input would look like. 

We recommend opening the new `article_guideline.md` from `02_sample_medium` and comparing it to the one from `01_sample_small`.


In [27]:
SAMPLE_DIR = Path("inputs/tests/02_sample_medium")
SAMPLE_DIR.exists()

True

Now, let's get to it. We will start everything from scratch for complete clarity.

### Step 1: Load all necessary context

In [28]:
from brown.loaders import (
    MarkdownArticleExampleLoader,
    MarkdownArticleGuidelineLoader,
    MarkdownArticleProfilesLoader,
    MarkdownResearchLoader,
)

pretty_print.wrapped("STEP 1: Loading Context", width=100)

# Load guideline
guideline_loader = MarkdownArticleGuidelineLoader(uri=Path("article_guideline.md"))
article_guideline = guideline_loader.load(working_uri=SAMPLE_DIR)

# Load research
research_loader = MarkdownResearchLoader(uri=Path("research.md"))
research = research_loader.load(working_uri=SAMPLE_DIR)

# Load profiles
profiles_input = {
    "article": PROFILES_DIR / "article_profile.md",
    "character": PROFILES_DIR / "character_profiles" / "paul_iusztin.md",
    "mechanics": PROFILES_DIR / "mechanics_profile.md",
    "structure": PROFILES_DIR / "structure_profile.md",
    "terminology": PROFILES_DIR / "terminology_profile.md",
    "tonality": PROFILES_DIR / "tonality_profile.md",
}
profiles_loader = MarkdownArticleProfilesLoader(uri=profiles_input)
article_profiles = profiles_loader.load()

# Load examples
examples_loader = MarkdownArticleExampleLoader(uri=EXAMPLES_DIR)
article_examples = examples_loader.load()

print(f"✓ Guideline: {len(article_guideline.content):,} characters")
print(f"✓ Research: {len(research.content):,} characters, {len(research.image_urls)} images")
print(f"✓ Profiles: {len(profiles_input)} profiles loaded")
print(f"✓ Examples: {len(article_examples.examples)} article examples")

----------------------------------------------------------------------------------------------------
  STEP 1: Loading Context
----------------------------------------------------------------------------------------------------
✓ Guideline: 22,854 characters
✓ Research: 211,790 characters, 14 images
✓ Profiles: 6 profiles loaded
✓ Examples: 2 article examples


### Step 2: Generate media items

In [29]:
import asyncio

from brown.entities.media_items import MediaItems
from brown.models import SupportedModels, get_model
from brown.nodes import MediaGeneratorOrchestrator, MermaidDiagramGenerator, Toolkit

# Create worker tool
diagram_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
mermaid_generator = MermaidDiagramGenerator(model=diagram_model)
toolkit = Toolkit(tools=[mermaid_generator.as_tool()])

# Create orchestrator
orchestrator_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
orchestrator = MediaGeneratorOrchestrator(
    article_guideline=article_guideline,
    research=research,
    model=orchestrator_model,
    toolkit=toolkit,
)

# Get media generation jobs
pretty_print.wrapped("Analyzing article guideline for media requirements...")
media_jobs = await orchestrator.ainvoke()


pretty_print.wrapped("Found {len(media_jobs)} media items to generate")
media_jobs_dict = {}
for i, job in enumerate(media_jobs):
    pretty_print.wrapped(
        {
            "Tool": job["name"],
            "Description": job["args"].get("description_of_the_diagram", "N/A")[:100] + "...",
            "Section": job["args"].get("section_title", "N/A"),
        },
        title=f"Job {i + 1}",
    )

pretty_print.wrapped("Generating media items in parallel...")
coroutines = []
for job in media_jobs:
    tool = orchestrator.toolkit.get_tool_by_name(job["name"])
    if tool:
        coroutines.append(tool.ainvoke(job["args"]))

media_items = await asyncio.gather(*coroutines)
media_items = MediaItems.build(media_items=media_items)

pretty_print.wrapped(f"Generated {len(media_items.media_items)} media items successfully!")

----------------------------------------------------------------------------------------------------
  Analyzing article guideline for media requirements...
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
  Found {len(media_jobs)} media items to generate
----------------------------------------------------------------------------------------------------
---------------------------------------------- Job 1 ----------------------------------------------
  {
  "Tool": "mermaid_diagram_generator_tool",
  "Description": "A flowchart illustrating a simple LLM workflow. The workflow starts, an LLM call or other operation ...",
  "Section": "Understanding the Spectrum: From Workflows to Agents"
}
----------------------------------------------------------------------------------------------------
---------------------------------------------- 

As context objects:

In [30]:
pretty_print.wrapped(f"{media_items.to_context()}", title="Media Items As Context")

-------------------------------------- Media Items As Context --------------------------------------
  
<media_items>

<mermaid_diagram>
    <location>Understanding the Spectrum: From Workflows to Agents</location>
    <content>```mermaid
graph TD
    A["Start"] --> B["LLM Call / Data Operation"]
    B --> C["End"]
```</content>
    <caption>A simple LLM workflow flowchart.</caption>
</mermaid_diagram>


<mermaid_diagram>
    <location>Choosing Your Path</location>
    <content>```mermaid
graph TD
    A["AI Generates Content/Solution"]
    B["Human Reviews and Verifies Output"]
    A --> B
    B --> A: "Refinement/Correction Feedback"
```</content>
    <caption>A flowchart illustrating the AI generation and human verification loop with a feedback mechanism.</caption>
</mermaid_diagram>


<mermaid_diagram>
    <location>Exploring Common Patterns</location>
    <content>```mermaid
graph TD
    A["Start"] --> B{"LLM Router Decision"}
    B -->|"Route to Chain 1"| C["LLM Chain 1"]
    B --

### Step 3: Generate article

In [31]:
from brown.nodes import ArticleWriter

pretty_print.wrapped("STEP 3: Writing Article (First Draft)", width=100)
print("This may take 1-2 minutes...")

writer_model = get_model(SupportedModels.GOOGLE_GEMINI_25_FLASH)
article_writer = ArticleWriter(
    article_guideline=article_guideline,
    research=research,
    article_profiles=article_profiles,
    media_items=media_items,
    article_examples=article_examples,
    model=writer_model,
)

article = await article_writer.ainvoke()

print(f"✓ Article generated: {len(article.content):,} characters")
pretty_print.wrapped(article.content[:1000], title="First Draft (First 1000 chars)", width=120)

----------------------------------------------------------------------------------------------------
  STEP 3: Writing Article (First Draft)
----------------------------------------------------------------------------------------------------
This may take 1-2 minutes...
✓ Article generated: 26,658 characters
-------------------------------------------- First Draft (First 1000 chars) --------------------------------------------
  # Workflows vs. Agents: The AI Engineering Decision That Shapes Success
### Choose wisely to ship reliable, cost-effective AI applications

When building AI applications, we often face a critical architectural decision early on: do we create a predictable, step-by-step workflow where we control every action, or do we build an autonomous agent that can think and decide for itself? This is one of the key choices that impacts everything from development time and costs to reliability and user experience. Making the wrong move can lead to an overly rigid system that

Even the article is a context object:

In [32]:
pretty_print.wrapped(f"{article.to_context()[:1500]}...", title="Generated Article As Context")

----------------------------------- Generated Article As Context -----------------------------------
  
<article>
    # Workflows vs. Agents: The AI Engineering Decision That Shapes Success
### Choose wisely to ship reliable, cost-effective AI applications

When building AI applications, we often face a critical architectural decision early on: do we create a predictable, step-by-step workflow where we control every action, or do we build an autonomous agent that can think and decide for itself? This is one of the key choices that impacts everything from development time and costs to reliability and user experience. Making the wrong move can lead to an overly rigid system that breaks with unexpected user input or new features. It can also result in an unpredictable agent that shines in demos but fails catastrophically when it matters most.

We have seen, especially in 2024 and 2025, how this architectural decision can make or break billion-dollar AI startups. Successful companies, team

### Step 4: Save the articles

In [33]:
from brown.renderers import MarkdownArticleRenderer

output_path = SAMPLE_DIR / "article.md"
renderer = MarkdownArticleRenderer()
renderer.render(article, output_uri=output_path)

Open it at the following path:

In [34]:
pretty_print.wrapped(f"Article saved to {output_path}", width=100)

----------------------------------------------------------------------------------------------------
  Article saved to inputs/tests/02_sample_medium/article.md
----------------------------------------------------------------------------------------------------


Now check out the article and let us know what you think!

## 10. Conclusion

Congratulations! In this lesson, we built the core engine of Brown, the writing workflow. We learned:

1. **Context Engineering:** How to map domain entities to XML-structured context using `ContextMixin` , plug them into a massive system prompt and help the LLM reason through them.
2. **Writing Profiles + Few-shot examples:** How to enforce style and voice using granular profile entities and few-shot examples.
3. **Orchestrator-Worker:** How to write the orchestrator-worker pattern from scratch and dynamically generate media items by delegating to specialized tools.

We now have a system that can write a good first draft. Still, because the system prompt is extremely complex, the LLM cannot reason through everything at once. In our opinion, that’s natural. Ultimately, even we, as humans, need a few iterations to refine our work.

That’s why, in the next lesson (Lesson 23), we will implement the **Evaluator-Optimizer** pattern to iteratively review and edit the article. Additionally, we will integrate everything into a robust LangGraph workflow. Then, in Lesson 24, we will expose these workflows via MCP to bring humans into the loop.

### Practicing Ideas

As a practical exercise for part 4 of the course, where you have to implement your own project, here are some ideas on how you can further extend the code:  

- **Add support for image and video generation** within the Orchestrator-Worker layer to create richer media assets.
- **Reduce costs and latency** by caching constant inputs (like research and profiles) between LLM calls to avoid recomputing them, and by compressing the research relative to the article guideline.
- **Extend the writer** to other media formats such as social media posts, email newsletter articles, technical documentation, or video transcripts.
- **Add different character profiles** alongside our Paul Iusztin one to support multiple voices and personas.

### Useful Resources

- **Brown Package**: Explore `lessons/writing_workflow`
- **Writing Profiles**: Check `inputs/profiles/` for more profile templates
- **Test Data**: Use `inputs/tests/` for additional examples

### 💡 Run Brown as a Standalone Python Project

Remember that you can also run `brown` as a standalone Python project by going to `lessons/writing_workflow/` and following the instructions from there.
